In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle, scipy, random, time
import matplotlib.pyplot as plt
%matplotlib inline
# plt.style.use('fivethirtyeight')

#import all necessary libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
import catboost, lightgbm, xgboost
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

## read data

In [393]:
f2 = pickle.load(open('feature_2.pickle', 'rb'))
f2.shape

(94, 529)

In [398]:
f2 = f2[['User_id', 'label'] + list(remained_cols)]
f2.shape

(94, 282)

In [406]:
df = pd.concat([df, f2], axis = 0)

In [407]:
df.shape

(130443, 282)

In [401]:
df.columns

Index(['User_id', 'label', 'wave_appro_x_1', 'wave_appro_x_2',
       'wave_appro_x_3', 'wave_appro_y_1', 'wave_appro_y_2', 'wave_appro_y_3',
       'wave_appro_z_1', 'wave_appro_z_2',
       ...
       'gra_x_velocity', 'gra_y_velocity', 'gra_z_velocity',
       'gra_mag_velocity', 'body_y_velocity', 'body_mag_velocity',
       'gra_x_y_rela', 'gra_x_z_rela', 'gra_y_z_rela', 'gra_x_y_covar'],
      dtype='object', length=282)

In [425]:
# pickle.dump(df, open('final_data.pkl', 'wb'))

In [2]:
df = pickle.load(open('final_data.pkl', 'rb'))
df.dropna(inplace=True)
df.drop(columns=['User_id'], inplace=True)
df['label'] = df['label'].astype('int')
# df.drop(columns=list(del_cols), inplace=True)
df.shape

(130443, 281)

In [434]:
# X, y = df.iloc[:, 1:], df['label']

In [3]:
df['label'].value_counts()

4     43712
2     34661
14    18119
12     9774
9      6410
10     5031
7      2812
16     2448
13     2440
19     1722
11     1266
1       856
5       210
17      162
6       148
22      134
3       126
28      106
18       89
8        82
15       57
24       57
25       17
26        3
20        1
Name: label, dtype: int64

In [3]:
def under_sampling(df):
    labels = [4,2,14]
    n_rows = [33000, 24000, 8000]

    for label, rows in zip(labels, n_rows):
        ind = df[df['label']==label].index
        r = random.sample(range(0, len(ind)), rows)
        deleted = ind[r]
        df.drop(deleted, axis=0, inplace=True)
    print(df.shape)
    return df

In [4]:
def over_sampling(df):
    # !pip install imblearn
    from imblearn.over_sampling import RandomOverSampler
    os = RandomOverSampler(random_state=42)
    labels1 = df['label'].value_counts().head(11).index
    
    df1 = df[df['label'].isin(labels1)]
    labels2 = df['label'].value_counts().tail(14).index
    df2 = df[df['label'].isin(labels2)]
    
    X1, y1 = df1.drop(columns=['label']), df1['label']
    X2, y2 = os.fit_resample(df2.drop(columns=['label']), df2['label'])

    print(f'after undersampling(topclass data) : {X1.shape, y1.shape}')
    print(f'after oversampling(underclass data) : {X2.shape, y2.shape}')

    X = pd.concat([X1, X2], axis=0)
    y = pd.concat([y1, y2], axis=0)

    print(f'shape of my training data : {X.shape, y.shape}')
    return X,y

In [5]:
dff = under_sampling(df)
X, y = over_sampling(df)

dff = pd.concat([X,y], axis=1)
dff = dff.sample(frac=1).reset_index(drop=True)
X = dff.drop(columns=['label'])
y = dff['label']
print(X.shape, y.shape)
print()

(65399, 281)
after undersampling(topclass data) : ((63395, 280), (63395,))
after oversampling(underclass data) : ((11368, 280), (11368,))
shape of my training data : ((74763, 280), (74763,))
(74763, 280) (74763,)



In [6]:
pickle.dump()

4     10712
2     10661
14    10119
12     9774
9      6410
10     5031
7      2812
16     2448
13     2440
19     1722
11     1266
3       812
5       812
6       812
8       812
28      812
26      812
15      812
17      812
18      812
20      812
22      812
24      812
25      812
1       812
Name: label, dtype: int64

In [8]:
clf = lightgbm.LGBMClassifier(n_estimators=200, reg_lambda=0.01, reg_alpha=0.001).fit(X, y)

In [9]:
pickle.dump(clf, open('lgbm.pkl', 'wb'))

In [7]:
pickle.dump(X, open('X.pkl', 'wb'))
pickle.dump(y, open('y.pkl', 'wb'))

In [446]:
X = pickle.load(open('X.pkl', 'rb'))
y = pickle.load(open('y.pkl', 'rb'))

In [447]:
X.shape, y.shape

((74847, 280), (74847,))

In [448]:
y.value_counts()

4     10712
2     10661
14    10119
12     9774
9      6410
10     5031
7      2812
16     2448
13     2440
19     1722
11     1266
3       818
5       818
6       818
8       818
28      818
26      818
15      818
17      818
18      818
20      818
22      818
24      818
25      818
1       818
Name: label, dtype: int64

## model_train

In [449]:
def train_model(model):
    ini = time.time()
    clf = model.fit(X,y)
    cv_score = cross_val_score(clf, X, y, cv=5, scoring='accuracy')

#     print(f'model info for {model_name.upper()} classifier : ')
    print('---------------------------------------')
    print(f'Cross-validation : {cv_score}')
    print(f'mean : {np.mean(cv_score)}')
#     print(f'median : {np.median(cv_score)}')
#     print(f'std : {np.std(cv_score)}\n')
    print(f'shape of input data : {X.shape}')

#     train_score = clf.score(X_train, y_train)
#     test_score = clf.score(X_test, y_test)
    
#     print(f'train score : {train_score}')
#     print(f'test score : {test_score}')
#     final = time.time()
#     print(f'time required : {(final-ini)/60} minutes')    
#     model_info.append({
#         'cv_score': cv_score,
#         'model': model_name,
#         'mean_cv_score': np.mean(cv_score),
#         'median': np.median(cv_score)
# #         'train_score': train_score,
# #         'test_score': test_score,
#     })
#     return clf

In [307]:
clf = train_model(ExtraTreesClassifier(n_estimators=200, max_depth=12, criterion='entropy', n_jobs=-1))

---------------------------------------
Cross-validation : [0.83074336 0.82953246 0.83221206 0.83389397 0.83261572]
mean : 0.8317995118969904
shape of input data : ((74322, 525), (74322,))
time required : 4.1030228217442835 minutes


In [149]:
train_model(RandomForestClassifier(n_estimators=400, max_depth=14, n_jobs=-1))

---------------------------------------
Cross-validation : [0.85482677 0.85967037 0.8556916  0.85394241 0.85811356]
mean : 0.8564489437951973
median : 0.8556916038751345
shape of input data : ((74322, 280), (74322,))
time required : 30.606165953477223 minutes


In [378]:
train_model(RandomForestClassifier(n_estimators=600, max_depth=13, n_jobs=-1, criterion='entropy'))

---------------------------------------
Cross-validation : [0.87350835 0.87719059 0.87397709 0.87602291 0.87370431]
mean : 0.874880650587247
shape of input data : (73322, 280)


In [363]:
train_model(lightgbm.LGBMClassifier(n_estimators=200, reg_lambda=0.001, reg_alpha=0.1, max_depth=4))

---------------------------------------
Cross-validation : [0.87241732 0.87241732 0.87397709 0.87697763 0.87343153]
mean : 0.8738441784691722
shape of input data : (73322, 280)


In [362]:
lightgbm.LGBMClassifier?

In [349]:
train_model(lightgbm.LGBMClassifier(n_estimators=200, reg_alpha=0.1))

---------------------------------------
Cross-validation : [0.8718718  0.86866689 0.87370431 0.87404528 0.87152209]
mean : 0.8719620766680635
median : 0.871871803614047
shape of input data : ((73322, 280), (73322,))
time required : 23.804763889312746 minutes


In [156]:
# def hyperopt_random(model, param_grid):
#     clf = RandomizedSearchCV(model, param_grid, cv=5, verbose=2, n_jobs=-1,
# return_train_score=False, n_iter=5)
#     clf.fit(X, y)
    
#     print(pd.DataFrame(clf.cv_results_)[['params','mean_test_score']].sort_values(by=
#     ['mean_test_score'], ascending=False).head(3))
#     print(clf.best_params_)

In [155]:
# hyperopt_random(ExtraTreesClassifier(n_jobs=-1),{
#  'criterion': ['gini', 'entropy'],
#  'max_depth': [13, 14, 15],
#  'max_features': ['auto', 'sqrt'],
#  'n_estimators': [100, 200, 500, 700, 800]
# })

# Feature selection 

In [313]:
clf.feature_importances_

array([4.40820658e-03, 1.34596703e-04, 4.84336101e-03, 2.47882348e-04,
       7.01349666e-03, 2.29987972e-04, 2.57508595e-03, 7.88206535e-05,
       2.61133199e-03, 8.85469210e-05, 3.74828080e-03, 7.42379187e-05,
       1.13638478e-02, 7.59843391e-05, 7.42173623e-03, 8.84027112e-05,
       1.19636286e-02, 7.06437201e-05, 1.50624519e-03, 7.46564447e-05,
       1.82859564e-03, 1.47992052e-04, 2.97605833e-03, 6.68621975e-05,
       1.28235794e-04, 1.20603988e-04, 2.16541007e-04, 1.37851917e-04,
       1.01176981e-04, 1.14993359e-04, 1.19040014e-04, 8.92508846e-05,
       1.05656823e-04, 1.40831767e-04, 5.10519663e-04, 1.48527314e-04,
       8.56271260e-05, 9.96086148e-05, 1.06232192e-04, 9.38128193e-05,
       6.63343565e-04, 1.37247961e-03, 4.03332254e-04, 5.09759445e-03,
       8.75565660e-04, 8.60598297e-04, 9.51464078e-04, 1.28508215e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.07701870e-03, 1.93345955e-03, 1.46845083e-03, 1.24357066e-03,
      

In [317]:
treefi = pd.Series(clf.feature_importances_, index=X.columns).sort_values()
treefi

jerk_peak_frequency_z         0.000000
jerk_central_frequency_z      0.000000
jerk_central_frequency_mag    0.000000
jerk_peak_frequency_x         0.000000
jerk_peak_frequency_y         0.000000
                                ...   
gra_z_velocity                0.011766
x_mean                        0.011970
wave_appro_z_mean             0.012403
z_mean                        0.013004
wave_appro_z_max              0.013048
Length: 525, dtype: float64

In [318]:
pickle.dump(treefi, open('rf_fi.pkl', 'wb'))

In [299]:
from sklearn.feature_selection import mutual_info_classif
features = mutual_info_classif(X, y)
features

array([0.79823897, 0.1199256 , 0.81277713, 0.07569811, 0.85864306,
       0.08099838, 0.39145572, 0.087627  , 0.40159206, 0.04570755,
       0.47741928, 0.05174049, 0.93356935, 0.09345382, 0.93881644,
       0.08865256, 0.98947832, 0.09188892, 0.47030757, 0.08493257,
       0.47516033, 0.05378423, 0.54397317, 0.05593519, 0.30160423,
       0.31293192, 0.14692416, 0.34959285, 0.26832198, 0.27910491,
       0.1146114 , 0.27510294, 0.32125348, 0.32992915, 0.15509799,
       0.36356065, 0.06165777, 0.05471043, 0.05081728, 0.07133142,
       0.40324216, 0.42307964, 0.23648515, 0.46624748, 0.50402976,
       0.50753064, 0.49432283, 0.54968007, 0.00455221, 0.61100605,
       0.61302721, 0.61192961, 0.61494408, 0.61719485, 0.61599213,
       0.61590849, 0.61576595, 0.45509489, 0.45964453, 0.29582214,
       0.49317954, 0.40921684, 0.41294489, 0.24050579, 0.43386256,
       0.09970029, 0.08752098, 0.20123622, 0.11081912, 0.28010003,
       0.2686121 , 0.26475718, 0.27664009, 0.3042441 , 0.30697

In [302]:
data2 = pd.Series(features, index=X.columns).sort_values()
pickle.dump(data2, open('m_info.pkl', 'wb'))

In [301]:
data2

jerk_central_frequency_mag    0.004552
jerk_peak_frequency_y         0.007301
jerk_peak_frequency_mag       0.007528
jerk_body_y_mean              0.037856
jerk_body_x_mean              0.043701
                                ...   
gra_z_max                     1.025784
third_quartile_gra_x          1.026220
gra_x_max                     1.032593
third_quartile_z              1.055317
first_quartile_z              1.055781
Length: 520, dtype: float64

In [14]:
chi2, _ = scipy.stats.chisquare(df.drop(columns=['label']))

In [13]:
df.shape

(130349, 526)

In [17]:
chi2, _ = scipy.stats.chisquare(df.drop(columns=['label']))
b = pd.Series(chi2, index=df.drop(columns=['label']).columns).sort_values()
b

jerk_autcor_central_magnitude_mag   -2.197518e+10
body_x_velocity                     -2.031595e+10
body_x_mean                         -2.030430e+10
jerk_autcor_central_magnitude_z     -1.596884e+10
jerk_autcor_central_magnitude_x     -1.509798e+10
                                         ...     
jerk_body_x_y_covar                  5.699489e+09
body_z_velocity                      5.797312e+09
jerk_z_var                           6.687835e+09
jerk_autcor_peak_mag_x               1.186011e+10
jerk_x_var                           1.255702e+10
Length: 525, dtype: float64

In [18]:
pickle.dump(a, open('chi2.pkl', 'wb'))

In [319]:
df.shape

(65349, 526)

In [8]:
from sklearn.feature_selection import f_classif
anova = f_classif(X, y)


2

In [9]:
anova, _ = anova

In [12]:
a = pd.Series(anova, index=X.columns).sort_values()
pickle.dump(a, open('anova.pkl', 'wb'))

In [7]:
X.drop(columns = X.iloc[:, [48, 49, 50, 96]].columns, inplace=True)

In [294]:
_, anova = anova
len(anova)

520

In [295]:
a = pd.Series(anova, index=X.columns).sort_values()
a

wave_appro_x_1                     0.000000
gra_y_median                       0.000000
gra_x_median                       0.000000
magnitude_median                   0.000000
z_median                           0.000000
                                     ...   
jerk_body_x_y_covar                0.695957
jerk_autcor_central_magnitude_y    0.703836
jerk_body_z_mean                   0.999287
jerk_body_y_mean                   1.000000
jerk_peak_frequency_y              1.000000
Length: 520, dtype: float64

In [297]:
pickle.dump(var_thres, open('var_thres.pkl', 'wb'))

In [291]:
X.drop(columns=['jerk_peak_frequency_x'], inplace=True)

In [258]:
df

,label,wave_appro_x_1,wave_details_x_1,wave_appro_x_2,wave_details_x_2,wave_appro_x_3,wave_details_x_3,wave_appro_y_1,wave_details_y_1,wave_appro_y_2,wave_details_y_2,wave_appro_y_3,wave_details_y_3,wave_appro_z_1,wave_details_z_1,wave_appro_z_2,wave_details_z_2,wave_appro_z_3,wave_details_z_3,wave_appro_mag_1,wave_details_mag_1,wave_appro_mag_2,wave_details_mag_2,wave_appro_mag_3,wave_details_mag_3,body_fredo_mag_kurtosis_x,body_fredo_mag_kurtosis_y,body_fredo_mag_kurtosis_z,body_fredo_mag_kurtosis_mag,jerk_body_fredo_mag_kurtosis_x,jerk_body_fredo_mag_kurtosis_y,jerk_body_fredo_mag_kurtosis_z,jerk_body_fredo_mag_kurtosis_mag,body_fredo_mag_skew_x,body_fredo_mag_skew_y,body_fredo_mag_skew_z,body_fredo_mag_skew_mag,jerk_body_fredo_mag_skew_x,jerk_body_fredo_mag_skew_y,jerk_body_fredo_mag_skew_z,jerk_body_fredo_mag_skew_mag,body_fredo_mag_entropy_x,body_fredo_mag_entropy_y,body_fredo_mag_entropy_z,body_fredo_mag_entropy_mag,central_frequency_x,central_frequency_y,central_frequency_z,central_frequency_mag,jerk_central_frequency_mag,autcor_central_frequency_x,autcor_central_frequency_y,autcor_central_frequency_z,autcor_central_frequency_mag,jerk_autcor_central_frequency_x,jerk_autcor_central_frequency_y,jerk_autcor_central_frequency_z,jerk_autcor_central_frequency_mag,central_magnitude_x,central_magnitude_y,central_magnitude_z,central_magnitude_mag,jerk_central_magnitude_x,jerk_central_magnitude_y,jerk_central_magnitude_z,jerk_central_magnitude_mag,autcor_central_magnitude_x,autcor_central_magnitude_y,autcor_central_magnitude_z,autcor_central_magnitude_mag,jerk_autcor_central_magnitude_x,jerk_autcor_central_magnitude_y,jerk_autcor_central_magnitude_z,jerk_autcor_central_magnitude_mag,no_of_peaks_x,no_of_peaks_y,no_of_peaks_z,no_of_peaks_mag,jerk_no_of_peaks_x,jerk_no_of_peaks_y,jerk_no_of_peaks_z,jerk_no_of_peaks_mag,autcor_no_of_peaks_x,autcor_no_of_peaks_y,autcor_no_of_peaks_z,autcor_no_of_peaks_mag,jerk_autcor_no_of_peaks_x,jerk_autcor_no_of_peaks_y,jerk_autcor_no_of_peaks_z,jerk_autcor_no_of_peaks_mag,peak_frequency_x,peak_frequency_y,peak_frequency_z,peak_frequency_mag,jerk_peak_frequency_x,jerk_peak_frequency_y,jerk_peak_frequency_mag,autcor_peak_frequency_x,autcor_peak_frequency_y,autcor_peak_frequency_z,autcor_peak_frequency_mag,jerk_autcor_peak_frequency_x,jerk_autcor_peak_frequency_y,jerk_autcor_peak_frequency_z,jerk_autcor_peak_frequency_mag,peak_mag_x,peak_mag_y,peak_mag_z,peak_mag_mag,jerk_peak_mag_x,jerk_peak_mag_y,jerk_peak_mag_z,jerk_peak_mag_mag,autcor_peak_mag_x,autcor_peak_mag_y,autcor_peak_mag_z,autcor_peak_mag_mag,jerk_autcor_peak_mag_x,jerk_autcor_peak_mag_y,jerk_autcor_peak_mag_z,jerk_autcor_peak_mag_mag,first_5_fft_co_x1,first_5_fft_co_x2,first_5_fft_co_x3,first_5_fft_co_x4,first_5_fft_co_x5,first_5_fft_co_y1,first_5_fft_co_y2,first_5_fft_co_y3,first_5_fft_co_y4,first_5_fft_co_y5,first_5_fft_co_z1,first_5_fft_co_z2,first_5_fft_co_z3,first_5_fft_co_z4,first_5_fft_co_z5,first_5_fft_co_mag1,first_5_fft_co_mag2,first_5_fft_co_mag3,first_5_fft_co_mag4,first_5_fft_co_mag5,jerk_first_5_fft_co_x1,jerk_first_5_fft_co_x2,jerk_first_5_fft_co_x3,jerk_first_5_fft_co_x4,jerk_first_5_fft_co_x5,jerk_first_5_fft_co_y1,jerk_first_5_fft_co_y2,jerk_first_5_fft_co_y3,jerk_first_5_fft_co_y4,jerk_first_5_fft_co_y5,jerk_first_5_fft_co_z1,jerk_first_5_fft_co_z2,jerk_first_5_fft_co_z3,jerk_first_5_fft_co_z4,jerk_first_5_fft_co_z5,jerk_first_5_fft_co_mag1,jerk_first_5_fft_co_mag2,jerk_first_5_fft_co_mag3,jerk_first_5_fft_co_mag4,jerk_first_5_fft_co_mag5,acc_x_rms,acc_y_rms,acc_z_rms,acc_mag_rms,gra_x_rms,gra_y_rms,gra_z_rms,gra_mag_rms,body_x_rms,body_y_rms,body_z_rms,body_mag_rms,jerk_x_rms,jerk_y_rms,jerk_z_rms,jerk_mag_rms,jerk_body_x_rms,jerk_body_y_rms,jerk_body_z_rms,jerk_body_mag_rms,wave_appro_x_rms,wave_appro_y_rms,wave_appro_z_rms,wave_appro_mag_rms,wave_deltail_x_rms,wave_deltail_y_rms,wave_deltail_z_rms,wave_deltail_mag_rms,x_min,y_min,z_min,mag_min,gra_x_min,gra_y_min,gra_z_min,gra_mag_min,body_x_min,body_y_min,body_z_m

In [260]:
from sklearn.feature_selection import VarianceThreshold
vthres = VarianceThreshold(threshold=0.8)
vthres.fit(df)
vthres

VarianceThreshold(threshold=0)

In [275]:
var = []
for col in X.columns:
    var.append((np.std(X[col]))**2)

In [279]:
var_thres = pd.Series(var, index=X.columns).sort_values()
pickle.dump(var_thres, open('var_thres', 'wb'))

## split users

In [481]:
train_df = df[df['User_id'].isin([2,3,4,5,6,12,17])]
X_train, y_train = train_df.drop(columns=['User_id', 'label']), train_df['label']
print(train_df.shape)
print(X_train.shape, y_train.shape)

(98975, 282)
(98975, 280) (98975,)


In [482]:
val_df = df[~df['User_id'].isin([2,3,4,5,6,12,17])]
X_val, y_val = val_df.drop(columns=['User_id', 'label']), val_df['label']
print(val_df.shape)
print(X_val.shape, y_val.shape)

(31468, 282)
(31468, 280) (31468,)


In [483]:
train_df = train_df.drop(columns=['User_id'])
val_df = val_df.drop(columns=['User_id'])

In [469]:
def under_sampling(df):
    labels = [4,2]
    n_rows = [28000, 5000]

    for label, rows in zip(labels, n_rows):
        ind = df[df['label']==label].index
        r = random.sample(range(0, len(ind)), rows)
        deleted = ind[r]
        df.drop(deleted, axis=0, inplace=True)
    print(df.shape)
    return df

def over_sampling(df):
    # !pip install imblearn
    from imblearn.over_sampling import RandomOverSampler
    os = RandomOverSampler(random_state=42)
    labels1 = df['label'].value_counts().head(9).index
    
    df1 = df[df['label'].isin(labels1)]
    labels2 = df['label'].value_counts().tail(16).index
    df2 = df[df['label'].isin(labels2)]
    
    X1, y1 = df1.drop(columns=['label']), df1['label']
    X2, y2 = os.fit_resample(df2.drop(columns=['label']), df2['label'])

    print(f'after undersampling(topclass data) : {X1.shape, y1.shape}')
    print(f'after oversampling(underclass data) : {X2.shape, y2.shape}')

    X = pd.concat([X1, X2], axis=0)
    y = pd.concat([y1, y2], axis=0)

    print(f'shape of my training data : {X.shape, y.shape}')
    return X,y

In [470]:
dff = under_sampling(train_df)
X, y = over_sampling(train_df)

dff = pd.concat([X,y], axis=1)
dff = dff.sample(frac=1).reset_index(drop=True)
X = dff.drop(columns=['label'])
y = dff['label']
print(X.shape, y.shape)

(65963, 281)
after undersampling(topclass data) : ((63628, 280), (63628,))
after oversampling(underclass data) : ((12320, 280), (12320,))
shape of my training data : ((75948, 280), (75948,))
(75948, 280) (75948,)


In [389]:
# for lmbda in [0.001, 0.01, 0.1]:
#     lgbm_model(lmbda)

In [478]:
# def lgbm_model(lmbda):
# lambda=0.01, alpha=0.001
ini = time.time()
clf = lightgbm.LGBMClassifier(n_estimators=200, reg_lambda=0.01, reg_alpha=0.001).fit(X, y)
# print(f'alpha={lmbda} : ')
final = time.time()
print(f'time required : {(final-ini)/60} minutes')
print(f'train score : {clf.score(X, y)}')
print(f'validation score : {clf.score(X_val, y_val)}')
print()

time required : 5.246815745035807 minutes
train score : 0.9824616843103176
validation score : 0.5187174272276599



In [388]:
# def lgbm_model(lmbda):
ini = time.time()
clf = RandomForestClassifier(n_estimators=600, max_depth=13, n_jobs=-1, criterion='entropy').fit(X, y)
# RandomForestClassifier(n_estimators=600, max_depth=13, n_jobs=-1, criterion='entropy')
print(f'alpha={lmbda} : ')
final = time.time()
print(f'time required : {(final-ini)/60} minutes')
print(f'train score : {clf.score(X, y)}')
print(f'validation score : {clf.score(X_val, y_val)}')
print()

alpha=0.1 : 
time required : 24.767437314987184 minutes
train score : 0.9305097874463364
validation score : 0.3422818791946309

